In [ ]:
!pip install PyYAML

In [1]:
import cv2
import os
import yaml

import numpy as np

from yaml.loader import SafeLoader
from yolo_predictions import YOLO_Pred

In [ ]:
# load YAML
with open('data.yaml', mode='r') as f:
    data_yaml = yaml.load(f, Loader=SafeLoader)

labels = data_yaml['names']
labels

In [ ]:
# load model
yolo = cv2.dnn.readNetFromONNX('./Model2/weights/best.onnx')
yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [ ]:
# load image
img = cv2.imread('./test_ds/091db72ac0aef55b790ffc651f4eef6f.jpg')
image = img.copy()
row, col, d = image.shape

# Get predictions from the image
# step-1 convert image into square image (array)
max_rc = max(row, col)
input_image = np.zeros((max_rc, max_rc, 3), dtype=np.uint8)
input_image[0:row, 0:col] = image

# step-2 get prediction from square array
INPUT_WH_YOLO = 640
blob = cv2.dnn.blobFromImage(input_image, 1/255, (INPUT_WH_YOLO, INPUT_WH_YOLO), swapRB=True, crop=False)
yolo.setInput(blob)
preds = yolo.forward() # detection or prediction from model

In [ ]:
image.shape

In [ ]:
preds.shape

In [ ]:
# Non-Max Suppression (NMS)
# step-1: filter detection based on confidence score (0.4)
# and probability score (0.25)
detections = preds[0]
boxes = []
confidences = []
classes = []

# calculate width & height of image (input_image)
image_w, image_h = input_image.shape[:2]
x_factor = image_w/INPUT_WH_YOLO
y_factor = image_h/INPUT_WH_YOLO

for i in range(len(detections)):
    row = detections[i]
    confidence = row[4] # confidence of obj detection
    if confidence > 0.1:
        class_score = row[5:].max() # max prob from 21 obj classes
        class_id = row[5:].argmax() # get idx position at which max prob occur

        if class_score > 0.1:
            cx, cy, w, h = row[0:4]
            
            # construct bbox from four values
            # left, top, width, height
            left = int((cx - 0.5 * w) * x_factor)
            top = int((cy - 0.5 * h) * y_factor)
            width = int(w * x_factor)
            height = int(h * y_factor)

            box = np.array([left, top, width, height])

            # append values into the list
            confidences.append(confidence)
            boxes.append(box)
            classes.append(class_id)

In [ ]:
confidences

In [ ]:
# clean
boxes_np = np.array(boxes).tolist()
confidences_np = np.array(confidences).tolist()

# NMS
index = cv2.dnn.NMSBoxes(boxes_np, confidences_np, 0.10, 0.10).flatten()

In [ ]:
index

In [ ]:
len(index)

In [ ]:
# Draw Bounding Box
for ind in index:
    # extract bbox
    x, y, w, h = boxes_np[ind]
    bb_conf = int(confidences_np[ind] * 100)
    classes_id = classes[ind]
    class_name = labels[classes_id]

    text = f'{class_name}: {bb_conf}%'

    # draw bbox
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv2.rectangle(image, (x, y - 30), (x + w, y), (255, 255, 255), -1)
    cv2.putText(image, text, (x, y - 10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 0, 0), 1)

In [ ]:
cv2.imwrite('prediction_image_4.jpg', image)

Video Prediction

In [2]:
yolo = YOLO_Pred('./Model2/weights/best.onnx','data.yaml')

In [3]:
cap = cv2.VideoCapture('./video/demo_vid.MP4')

while True:
    ret, frame = cap.read()
    if ret == False:
        print('unable to read video')
        break
    pred_image = yolo.predictions(frame)
    cv2.imshow('YOLO', pred_image)
    if cv2.waitKey(1) == 27:
        break

cv2.destroyAllWindows()
cap.release()

unable to read video


: 